In [33]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb
from xgboost import XGBClassifier

In [34]:
# reading dataset using panda
data_path = os.path.dirname(os.getcwd())
data_df = pd.read_csv(os.path.join(data_path, "data\\data_income.csv"))

In [35]:
data_df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [36]:
# Define the features and target
X = data_df.drop(["income"], axis=1)
y = data_df["income"]

# Define the categorical columns to one-hot encode
categorical_cols = [
    "occupation",
    "education",
    "workclass",
    "marital-status",
    "relationship",
    "race",
    "native-country",
    "gender",
]
numeric_cols = [
    "age",
    "fnlwgt",
    "educational-num",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [37]:
# fix the format for y
y_train = y_train.replace({">50K": 1, "<=50K": 0})
y_test = y_test.replace({">50K": 1, "<=50K": 0})


In [38]:
# Define preprocessing for categorical and numeric data
categorical_preprocessor = OneHotEncoder()
numeric_preprocessor = StandardScaler()

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_preprocessor, categorical_cols),
        ("num", numeric_preprocessor, numeric_cols),
    ],
    remainder="passthrough",  # You can use this to include any remaining columns
)

# Create a pipeline with preprocessing and the machine learning model
model = XGBClassifier(random_state=42)

pipeline = Pipeline([("preprocessor", preprocessor), ("model", model)])
# Fit the pipeline (including preprocessing) to the training data
pipeline.fit(X_train, y_train)
# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8775


Plot de l'accuracy par genre et couleur de peau

In [39]:
from sklearn.metrics import accuracy_score

# Calculer l'accuracy par genre
accuracy_par_genre = {}
genres = ['Male', 'Female']
for genre in genres:
    mask_genre = X_test['gender'] == genre
    accuracy_genre = accuracy_score(y_test[mask_genre], y_pred[mask_genre])
    accuracy_par_genre[genre] = accuracy_genre

# Calculer l'accuracy par race
accuracy_par_race = {}
races = ['White', 'Black']
for race in races:
    mask_race = X_test['race'] == race
    accuracy_race = accuracy_score(y_test[mask_race], y_pred[mask_race])
    accuracy_par_race[race] = accuracy_race

# Créer un tableau double entrée avec les accuracies
tableau_accuracies = pd.DataFrame({
    'Global': accuracy,
    'Male': accuracy_par_genre['Male'],
    'Female': accuracy_par_genre['Female'],
    'White': accuracy_par_race['White'],
    'Black': accuracy_par_race['Black']
}, index=['Accuracy'])

print(tableau_accuracies)

           Global      Male   Female     White     Black
Accuracy  0.87747  0.844724  0.94291  0.870053  0.935103


Calcul des TPR (True positive rates) pour chaque catégorie

In [40]:
from sklearn.metrics import confusion_matrix

# Calculer la matrice de confusion pour chaque catégorie
conf_matrix_overall = confusion_matrix(y_test, y_pred)
conf_matrix_male = confusion_matrix(y_test[X_test['gender'] == 'Male'], y_pred[X_test['gender'] == 'Male'])
conf_matrix_female = confusion_matrix(y_test[X_test['gender'] == 'Female'], y_pred[X_test['gender'] == 'Female'])
conf_matrix_white = confusion_matrix(y_test[X_test['race'] == 'White'], y_pred[X_test['race'] == 'White'])
conf_matrix_black = confusion_matrix(y_test[X_test['race'] == 'Black'], y_pred[X_test['race'] == 'Black'])

# Calculer les taux de vrais positifs
tp_rate_overall = conf_matrix_overall[1, 1] / (conf_matrix_overall[1, 1] + conf_matrix_overall[1, 0])
tp_rate_male = conf_matrix_male[1, 1] / (conf_matrix_male[1, 1] + conf_matrix_male[1, 0])
tp_rate_female = conf_matrix_female[1, 1] / (conf_matrix_female[1, 1] + conf_matrix_female[1, 0])
tp_rate_white = conf_matrix_white[1, 1] / (conf_matrix_white[1, 1] + conf_matrix_white[1, 0])
tp_rate_black = conf_matrix_black[1, 1] / (conf_matrix_black[1, 1] + conf_matrix_black[1, 0])

# Créer un tableau double entrée avec les taux de vrais positifs
tableau_tp_rates = pd.DataFrame({
    'Overall': [tp_rate_overall],
    'Male': [tp_rate_male],
    'Female': [tp_rate_female],
    'White': [tp_rate_white],
    'Black': [tp_rate_black]
}, index=['True Positive Rate'])

# Afficher le tableau
print(tableau_tp_rates)

                     Overall     Male    Female     White     Black
True Positive Rate  0.676419  0.68932  0.600601  0.679101  0.596491


False negative rates

In [41]:
# Calculer les taux de faux négatifs
fn_rate_overall = conf_matrix_overall[1, 0] / (conf_matrix_overall[1, 0] + conf_matrix_overall[1, 1])
fn_rate_male = conf_matrix_male[1, 0] / (conf_matrix_male[1, 0] + conf_matrix_male[1, 1])
fn_rate_female = conf_matrix_female[1, 0] / (conf_matrix_female[1, 0] + conf_matrix_female[1, 1])
fn_rate_white = conf_matrix_white[1, 0] / (conf_matrix_white[1, 0] + conf_matrix_white[1, 1])
fn_rate_black = conf_matrix_black[1, 0] / (conf_matrix_black[1, 0] + conf_matrix_black[1, 1])

# Créer un tableau double entrée avec les taux de faux négatifs
tableau_fn_rates = pd.DataFrame({
    'Overall': [fn_rate_overall],
    'Male': [fn_rate_male],
    'Female': [fn_rate_female],
    'White': [fn_rate_white],
    'Black': [fn_rate_black]
}, index=['False Negative Rate'])

# Afficher le tableau
print(tableau_fn_rates)

                      Overall     Male    Female     White     Black
False Negative Rate  0.323581  0.31068  0.399399  0.320899  0.403509


Intersection True Positive rates

In [42]:
# Calculer la matrice de confusion pour chaque catégorie
conf_matrix_overall = confusion_matrix(y_test, y_pred)
conf_matrix_white_male = confusion_matrix(y_test[(X_test['gender'] == 'Male') & (X_test['race'] == 'White')], y_pred[(X_test['gender'] == 'Male') & (X_test['race'] == 'White')])
conf_matrix_white_female = confusion_matrix(y_test[(X_test['gender'] == 'Female') & (X_test['race'] == 'White')], y_pred[(X_test['gender'] == 'Female') & (X_test['race'] == 'White')])
conf_matrix_black_male = confusion_matrix(y_test[(X_test['race'] == 'Black') & (X_test['gender'] == 'Male')], y_pred[(X_test['race'] == 'Black') & (X_test['gender'] == 'Male')])
conf_matrix_black_female = confusion_matrix(y_test[(X_test['race'] == 'Black') & (X_test['gender'] == 'Female')], y_pred[(X_test['race'] == 'Black') & (X_test['gender'] == 'Female')])

# Calculer les taux de vrais positifs
tp_rate_overall = conf_matrix_overall[1, 1] / (conf_matrix_overall[1, 1] + conf_matrix_overall[1, 0])
tp_rate_white_male = conf_matrix_white_male[1, 1] / (conf_matrix_white_male[1, 1] + conf_matrix_white_male[1, 0])
tp_rate_white_female = conf_matrix_white_female[1, 1] / (conf_matrix_white_female[1, 1] + conf_matrix_white_female[1, 0])
tp_rate_black_male = conf_matrix_black_male[1, 1] / (conf_matrix_black_male[1, 1] + conf_matrix_black_male[1, 0])
tp_rate_black_female = conf_matrix_black_female[1, 1] / (conf_matrix_black_female[1, 1] + conf_matrix_black_female[1, 0])

# Créer un tableau double entrée avec les taux de vrais positifs
tableau_tp_rates = pd.DataFrame({
    'Overall': [tp_rate_overall],
    'White Male': [tp_rate_white_male],
    'White Female': [tp_rate_white_female],
    'Black Male': [tp_rate_black_male],
    'Black Female': [tp_rate_black_female]
}, index=['True Positive Rate'])

# Afficher le tableau
print(tableau_tp_rates)

                     Overall  White Male  White Female  Black Male  \
True Positive Rate  0.676419     0.69098       0.60678    0.590909   

                    Black Female  
True Positive Rate      0.615385  


Taux de faux négatifs par groupes croisés

In [43]:
# Calculer les taux de faux négatifs
fn_rate_overall = conf_matrix_overall[1, 0] / (conf_matrix_overall[1, 0] + conf_matrix_overall[1, 1])
fn_rate_white_male = conf_matrix_white_male[1, 0] / (conf_matrix_white_male[1, 0] + conf_matrix_white_male[1, 1])
fn_rate_white_female = conf_matrix_white_female[1, 0] / (conf_matrix_white_female[1, 0] + conf_matrix_white_female[1, 1])
fn_rate_black_male = conf_matrix_black_male[1, 0] / (conf_matrix_black_male[1, 0] + conf_matrix_black_male[1, 1])
fn_rate_black_female = conf_matrix_black_female[1, 0] / (conf_matrix_black_female[1, 0] + conf_matrix_black_female[1, 1])

# Créer un tableau double entrée avec les taux de faux négatifs
tableau_fn_rates = pd.DataFrame({
    'Overall': [fn_rate_overall],
    'White Male': [fn_rate_white_male],
    'White Female': [fn_rate_white_female],
    'Black Male': [fn_rate_black_male],
    'Black Female': [fn_rate_black_female]
}, index=['False Negative Rate'])

# Afficher le tableau
print(tableau_fn_rates)

                      Overall  White Male  White Female  Black Male  \
False Negative Rate  0.323581     0.30902       0.39322    0.409091   

                     Black Female  
False Negative Rate      0.384615  


In [44]:
# Calculer les taux de vrais positifs à l'intersection des groupes
tp_rate_male_white = conf_matrix_male[1, 1] / (conf_matrix_male[1, 1] + conf_matrix_male[1, 0] + conf_matrix_white[1, 1] + conf_matrix_white[1, 0])
tp_rate_male_black = conf_matrix_male[1, 1] / (conf_matrix_male[1, 1] + conf_matrix_male[1, 0] + conf_matrix_black[1, 1] + conf_matrix_black[1, 0])
tp_rate_female_white = conf_matrix_female[1, 1] / (conf_matrix_female[1, 1] + conf_matrix_female[1, 0] + conf_matrix_white[1, 1] + conf_matrix_white[1, 0])
tp_rate_female_black = conf_matrix_female[1, 1] / (conf_matrix_female[1, 1] + conf_matrix_female[1, 0] + conf_matrix_black[1, 1] + conf_matrix_black[1, 0])

# Créer un tableau avec les taux de vrais positifs à l'intersection des groupes
tableau_tp_rates_intersection = pd.DataFrame({
    'Male & White': [tp_rate_male_white],
    'Male & Black': [tp_rate_male_black],
    'Female & White': [tp_rate_female_white],
    'Female & Black': [tp_rate_female_black]
}, index=['True Positive Rate Intersection'])

# Afficher le tableau
print(tableau_tp_rates_intersection)


                                 Male & White  Male & Black  Female & White  \
True Positive Rate Intersection      0.333251      0.651376        0.082508   

                                 Female & Black  
True Positive Rate Intersection        0.447427  


In [10]:
# fix the format for y
data_df['income'] = data_df['income'].replace({">50K": 1, "<=50K": 0})

Statistical Parity Test : Gender

In [32]:
from scipy.stats import chi2_contingency

# Faire des prédictions sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Créer un tableau de contingence pour le genre et la prédiction de revenu
contingency_table = pd.crosstab(X_test['gender'], y_pred, margins=True)

# Effectuer le test du chi-carré pour la parité statistique
chi2, p, _, _ = chi2_contingency(contingency_table.iloc[:-1, :-1])

# Calculer le degré de liberté
df = (contingency_table.shape[0] - 1) * (contingency_table.shape[1] - 1)

# Calculer la valeur critique à un niveau de signification donné (par exemple, 0,05)
alpha = 0.05
critical_value = chi2_contingency(contingency_table.iloc[:-1, :-1], correction=False)[1]
print("Valeur critique :", critical_value)
print("chi2 :", chi2)

# Vérifier si la statistique du chi-carré est supérieure à la valeur critique
if chi2 > critical_value:
    print("Il y a une disparité statistiquement significative par rapport au genre.")
else:
    print("Il n'y a pas de disparité statistiquement significative par rapport au genre.")

Valeur critique : 4.2460657987164824e-108
chi2 : 486.6479284916249
Il y a une disparité statistiquement significative par rapport au genre.


Statistical Parity Test : Race

In [31]:
# Faire des prédictions sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Créer un tableau de contingence pour le genre et la prédiction de revenu
contingency_table = pd.crosstab(X_test['race'], y_pred, margins=True)

# Effectuer le test du chi-carré pour la parité statistique
chi2, p, _, _ = chi2_contingency(contingency_table.iloc[:-1, :-1])

# Calculer le degré de liberté
df = (contingency_table.shape[0] - 1) * (contingency_table.shape[1] - 1)

# Calculer la valeur critique à un niveau de signification donné (par exemple, 0,05)
alpha = 0.05
critical_value = chi2_contingency(contingency_table.iloc[:-1, :-1], correction=False)[1]
print("Valeur critique :", critical_value)
print("chi2 :", chi2)

# Vérifier si la statistique du chi-carré est supérieure à la valeur critique
if chi2 > critical_value:
    print("Il y a une disparité statistiquement significative par rapport à la couleur de peau.")
else:
    print("Il n'y a pas de disparité statistiquement significative par rapport à la couleur de peau.")

Valeur critique : 1.0401903751083985e-24
chi2 : 118.64468203069983
Il y a une disparité statistiquement significative par rapport à la couleur de peau.


Conditional Statistical Parity Test : Gender

In [45]:
# Créer un DataFrame combinant les vraies étiquettes, les étiquettes prédites et l'attribut protégé (gender)
results_df = pd.DataFrame({'True_Labels': y_test, 'Predicted_Labels': y_pred, 'Gender': X_test['gender']})

# Calculer les comptes conditionnels pour CSP
csp_counts = results_df.groupby(['Gender', 'Predicted_Labels', 'True_Labels']).size().unstack(fill_value=0)

# Effectuer le test du chi-carré pour chaque groupe de genre
gender_groups = results_df['Gender'].unique()
chi2_values = {}

for gender_group in gender_groups:
    sub_df = csp_counts.loc[gender_group]
    chi2, p, _, _ = chi2_contingency(sub_df)
    chi2_values[gender_group] = chi2

# Calculer la valeur critique à un niveau de signification donné (par exemple, 0,05)
alpha = 0.05
df = (len(csp_counts.columns) - 1) * (len(csp_counts.index) - 1)

critical_value = chi2_contingency(sub_df, correction=False)[1]

# Vérifier si les statistiques du chi-carré pour tous les groupes de genre sont inférieures à la valeur critique
csp_satisfied = all(chi2 <= critical_value for chi2 in chi2_values.values())

print("chi2_values :", chi2_values)
print("critical_value :", critical_value)

if csp_satisfied:
    print("La parité statistique conditionnelle est satisfaite.")
else:
    print("La parité statistique conditionnelle n'est pas satisfaite.")

chi2_values : {'Female': 1408.063091067341, 'Male': 2509.2967865810433}
critical_value : 0.0
La parité statistique conditionnelle n'est pas satisfaite.


Conditional Statistical Parity Test : Race

In [46]:
# Créer un DataFrame combinant les vraies étiquettes, les étiquettes prédites et l'attribut protégé (gender)
results_df = pd.DataFrame({'True_Labels': y_test, 'Predicted_Labels': y_pred, 'Race': X_test['race']})

# Calculer les comptes conditionnels pour CSP
csp_counts = results_df.groupby(['Race', 'Predicted_Labels', 'True_Labels']).size().unstack(fill_value=0)

# Effectuer le test du chi-carré pour chaque groupe de genre
gender_groups = results_df['Race'].unique()
chi2_values = {}

for gender_group in gender_groups:
    sub_df = csp_counts.loc[gender_group]
    chi2, p, _, _ = chi2_contingency(sub_df)
    chi2_values[gender_group] = chi2

# Calculer la valeur critique à un niveau de signification donné (par exemple, 0,05)
alpha = 0.05
df = (len(csp_counts.columns) - 1) * (len(csp_counts.index) - 1)

critical_value = chi2_contingency(sub_df, correction=False)[1]

# Vérifier si les statistiques du chi-carré pour tous les groupes de genre sont inférieures à la valeur critique
csp_satisfied = all(chi2 <= critical_value for chi2 in chi2_values.values())

print("chi2_values :", chi2_values)
print("critical_value :", critical_value)

if csp_satisfied:
    print("La parité statistique conditionnelle est satisfaite.")
else:
    print("La parité statistique conditionnelle n'est pas satisfaite.")

chi2_values : {'White': 3423.061150354326, 'Other': 42.119691509691506, 'Black': 415.19767509902806, 'Asian-Pac-Islander': 97.72081321054685, 'Amer-Indian-Eskimo': 19.919112218530824}
critical_value : 1.8645325480191648e-07
La parité statistique conditionnelle n'est pas satisfaite.


Fairness Partial Dependance Plots

In [51]:
from fairlearn.metrics import demographic_parity_difference

# Attribut protégé
protected_attribute = 'gender'

# Définir la caractéristique d'intérêt et créer une grille de valeurs à analyser
feature_of_interest = 'income'
feature_values = np.linspace(data_df[feature_of_interest].min(), data_df[feature_of_interest].max(), num=20)

# Initialiser les listes pour stocker les mesures d'équité et l'exactitude du modèle
mesures_equite = []
accuracies = []

# Itérer sur les valeurs de la caractéristique et calculer les mesures d'équité
for value in feature_values:
    # Modifier le jeu de données pour fixer la caractéristique à la valeur actuelle
    modified_X_test = X_test.copy()
    modified_X_test[feature_of_interest] = value
    
    # Faire des prédictions sur le jeu de données modifié
    modified_y_pred = pipeline.predict(modified_X_test)
    
    # Calculer la mesure d'équité (Différence de parité démographique)
    mesure_equite = demographic_parity_difference(y_test, modified_y_pred, sensitive_features=X_test[protected_attribute])
    
    # Calculer l'exactitude du modèle
    accuracy = accuracy_score(y_test, modified_y_pred)
    
    # Ajouter la mesure d'équité et l'exactitude aux listes respectives
    mesures_equite.append(mesure_equite)
    accuracies.append(accuracy)

# Tracer le Fairness Partial Dependence Plot (FPDP)
plt.figure(figsize=(8, 6))
plt.plot(feature_values, mesures_equite, label='Mesure d\'Équité (DPD)', marker='o')
plt.plot(feature_values, accuracies, label='Exactitude du Modèle', marker='x')
plt.xlabel(feature_of_interest)
plt.ylabel('Mesure d\'Équité / Exactitude')
plt.title('Fairness Partial Dependence Plot')
plt.legend()
plt.grid(True)
plt.show()

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U5'), dtype('float64')) -> None